In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data

In [ ]:
# Check that the Stokes object file exists
stokes_filename = "generated/objects/stokes.pickle"
my_file = Path(stokes_filename)
if not my_file.is_file():
    print("Stokes object does not exists (", stokes_filename,"), please run file init_data.py first")

# Load the objects from the files
print("Loading Stokes data from file", stokes_filename)
stokes_list = pickle.load(open(stokes_filename, "rb"))

In [ ]:
# Calculate first derivative
stokes_list['I'].calc_first_derivative()

In [ ]:
# Calculate second derivative
stokes_list['I'].calc_second_derivative()

In [ ]:
# Plot derived Intensity
fig = stokes_list['I'].plot_frame_d(7)
fig = stokes_list['I'].plot_frame_d(25)

In [ ]:
# Plot derived Intensity
fig = stokes_list['I'].plot_frame_dd(7)
fig = stokes_list['I'].plot_frame_dd(25)

In [ ]:
plt.plot(stokes_list['I'].data_n[450,450,:])
plt.grid()

In [ ]:
plt.plot(stokes_list['I'].data_d[450,450,:],color='purple')
plt.grid()

In [ ]:
plt.plot(stokes_list['I'].data_dd[450,450,:],color='red')
plt.grid()

In [ ]:
# # Plot derived Intensity
# for i in range(117):
#     fig = stokes_list['I'].plot_frame_dd(i)

In [ ]:
# Calculate the vertical (longitudinal) component

#NOTE: these values are only for the first line
lambda0_1 = 6301.51  # in Angstroms
gbar_1 = 1.669 # Lozitsky
C1_1 =  4.6686 * 10**(-13) * lambda0_1**2 * gbar_1
f = 1 # filling factor, assumed to be 1 for now -- #TODO: should this be an array?

line_cuttoff = 55

# Bv = -np.sum(stokes_list['V'].data_n * stokes_list['I'].data_d, 2) / (C1 * f * np.sum(stokes_list['I'].data_d*stokes_list['I'].data_d, 2))
Bv_1 = -np.sum(stokes_list['V'].data_n[:,:,0:line_cuttoff] * stokes_list['I'].data_d[:,:,0:line_cuttoff], 2) / (C1_1 * f * np.sum(stokes_list['I'].data_d[:,:,0:line_cuttoff]**2, 2))

In [ ]:
plot_data(Bv_1, f'B longitudinal, first line', norm='linear')

In [ ]:
# Calculate the vertical (longitudinal) component

#NOTE: these values are only for the second line
lambda0_2 = 6302.50  # in Angstroms
gbar_2 = 2.487 # Lozitsky
C1_2 =  4.6686 * 10**(-13) * lambda0_2**2 * gbar_2

Bv_2 = -np.sum(stokes_list['V'].data_n[:,:,line_cuttoff:] * stokes_list['I'].data_d[:,:,line_cuttoff:], 2) / (C1_2 * f * np.sum(stokes_list['I'].data_d[:,:,line_cuttoff:]**2, 2))

In [ ]:
plot_data(Bv_2, f'B longitudinal, second line', norm='linear')

In [ ]:
# Calculate the horizontal (transverse) component

#NOTE: these values are only for the first line
Gbar_1 = gbar_1 #TODO
C2_1 = 5.4490 * 10**(-26) * lambda0_1**4 * Gbar_1
L = np.sqrt(stokes_list['Q'].data_n[:,:,:line_cuttoff]**2 + stokes_list['U'].data_n[:,:,:line_cuttoff]**2)
Bt_1 = np.sum(L * np.abs(stokes_list['I'].data_dd[:,:,:line_cuttoff]), 2) / (C2_1 * f * np.sum(np.abs(stokes_list['I'].data_dd[:,:,:line_cuttoff]), 2)**2)
Bt_1 = np.sqrt(Bt_1)

In [ ]:
plot_data(Bt_1, f'B transverse, first line')

In [ ]:
# Calculate the horizontal (transverse) component

#NOTE: these values are only for the second line
Gbar_2 = gbar_2 #TODO
C2_2 = 5.4490 * 10**(-26) * lambda0_2**4 * Gbar_2
L = np.sqrt(stokes_list['Q'].data_n[:,:,line_cuttoff:]**2 + stokes_list['U'].data_n[:,:,line_cuttoff:]**2)
Bt_2 = np.sum(L * np.abs(stokes_list['I'].data_dd[:,:,line_cuttoff:]), 2) / (C2_2 * f * np.sum(np.abs(stokes_list['I'].data_dd[:,:,:line_cuttoff]), 2)**2)
Bt_2 = np.sqrt(Bt_2)

In [ ]:
plot_data(Bt_2, f'B transverse, second line')

In [ ]:
# # Compute the angle
# tan2chi = np.sum(stokes_list['U'].data_n * stokes_list['I'].data_dd, 2) / np.sum(stokes_list['Q'].data_n * stokes_list['I'].data_dd, 2) 
# chi = np.arctan(tan2chi)/2

In [ ]:
# Compute the angle

# NOTE: valid only for the first line
tan2chi = np.sum(stokes_list['U'].data_n[:,:,:line_cuttoff] * stokes_list['I'].data_dd[:,:,:line_cuttoff], 2) / np.sum(stokes_list['Q'].data_n[:,:,:line_cuttoff] * stokes_list['I'].data_dd[:,:,:line_cuttoff], 2) 
chi = np.arctan(tan2chi)/2
chi2 = (np.arctan2(np.sum(stokes_list['U'].data_n[:,:,:line_cuttoff] * stokes_list['I'].data_dd[:,:,:line_cuttoff], 2), np.sum(stokes_list['Q'].data_n[:,:,:line_cuttoff] * stokes_list['I'].data_dd[:,:,:line_cuttoff], 2))) / 2


In [ ]:
chi2 = (np.arctan2(np.sum(stokes_list['U'].data_n * stokes_list['I'].data_dd, 2), np.sum(stokes_list['Q'].data_n * stokes_list['I'].data_dd, 2))) / 2

In [ ]:
# Plot one frame of data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
ax.set_xlabel('X axis (array index)')
ax.set_ylabel('Y axis (array index)')
ax.set_title('Chi')

img = ax.imshow(np.rad2deg(chi2), cmap='twilight')
    
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Angle [degrees]')


#TODO: why is the result only to +/- 90 degrees? /2 factor

In [ ]:
# Calculate inclination angle
lambda0 = 1 #TODO
gbar = 1 #TODO
Gbar = 1 #TODO
temp = np.sum(np.abs(stokes_list['I'].wave_array - lambda0) * L * stokes_list['V'].data_n**2 * stokes_list['I'].data_d, 2)
tan2gamma = 4/3 * gbar**2 / Gbar * temp / np.sum(np.abs(stokes_list['V'].data_n)**4)
gamma = np.arctan(np.sqrt(tan2gamma))

In [ ]:
# Plot one frame of data
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
ax.set_xlabel('X axis (array index)')
ax.set_ylabel('Y axis (array index)')
ax.set_title('Chi')

img = ax.imshow(np.rad2deg(gamma), cmap='twilight')
    
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Angle [degrees]')

In [ ]:
# gamma = np.cotan(tan2gamma)